# Pull Polygons from TerraMatch API

This notebook sets up the process to pull polygon geometries and metadata from the TerraMatch API. The steps for pulling polygons are as follows:
1. Set up configuration and API token
2. 

In [1]:
import requests
import yaml
import json
import pandas as pd
import geopandas as gpd
from tqdm import tqdm

## Set up token and API URL

In [2]:
# Set up token access
auth_path = '../secrets.yaml'
with open(auth_path) as auth_file:
    auth = yaml.safe_load(auth_file)
headers = {
    'Authorization': f"Bearer {auth['access_token']}"
    }

In [3]:
# TerraMatch API URLs
staging_url = "https://api-staging.terramatch.org/research/v3/sitePolygons?" # Use for testing queries
prod_url = "https://api.terramatch.org/research/v3/sitePolygons?" # Use to pull data for analysis

## API Request

In [4]:
def pull_tm_api_data(url, headers, project_ids):
    '''
    edits to the above function include:
        iterating through list of project ids within func so output is a df with 
        multiple projects
        add project id as a column to support maxar metadata request
        update to last record variable
        added tqdm progress bar  
    '''
    # List to store all retrieved polygon metadata
    results = []
    # Set up a progress bar
    with tqdm(total=len(project_ids), desc="Processing Projects", unit="project") as progress_bar:
        # For every project in the list of project_ids
        for project_id in project_ids:
            # Set parameters with the current project ID
            params = {
                'projectId[]': project_id,
                'polygonStatus[]': 'approved',
                'includeTestProjects': 'false',
                'page[size]': '100'
            }

            last_record = ''
            new_last_record = None  # Ensure it's defined before use

            while True:
                # Send GET request and store the response (polygon geometries & metadata)
                response = requests.get(url, headers=headers, params=params)

                # Check status code
                if response.status_code != 200:
                    raise ValueError(f'Request failed for project {project_id} with status code {response.status_code}')
                
                # Convert the response to a JSON and record the total number of records returned
                response_json = response.json()
                total_records = response_json['meta']['page']['total']

                # Parse response data
                # If there are no polygons for this project
                if total_records == 0:
                    break  # Exit if no data is available (skip to the next project)

                # Loop through each polygon in the response
                for idx in range(total_records):
                    # Extract polygon attributes from each record and store them in dictionary data
                    data = response_json['data'][idx]['attributes']
                    data['poly_id'] = response_json['data'][idx]['meta']['page']['cursor']
                    # Store the project_id in data
                    data['project_id'] = project_id 
                    # Append data ( a dictionary of that project's metadata) in the overall results list
                    results.append(data)

                    # Assign the last cursor only if there are records
                    if idx == (total_records - 1):
                        new_last_record = response_json['data'][idx]['meta']['page']['cursor']

                # Check if there are more pages
                if new_last_record and last_record != new_last_record:
                    last_record = new_last_record
                    params['page[after]'] = last_record
                else:
                    break  # Exit pagination if no new cursor is found

            progress_bar.update(1) 
    return results

In [5]:
# Create the list of projects to pull
full = pd.read_csv('../projects_all_approved_202501091214.csv')
full = full[(full.framework_key == 'terrafund-landscapes') | (full.framework_key == 'terrafund')]
full.framework_key.value_counts()

framework_key
terrafund               108
terrafund-landscapes     99
Name: count, dtype: int64

In [6]:
full

,project_id,project_name,organization_name,organisation_id,country,framework_key,description,status
0,c462918b-47f7-4ed5-99e0-7fec6e342036,"""Nakuru Eco-Reforestation Project""",NaN,1382,KE,terrafund-landscapes,MAIN ACTIVITIES\nJAN - FEB 2024- TREE NURSERY...,approved
1,caae56f9-0bb6-45a2-9d77-ff088b085917,0726 project,NaN,6283,BR,terrafund,org 0726,approved
2,c004619e-c1aa-4f7f-b56b-c8f9b4385d4e,1,NaN,1582,AL,terrafund,1,approved
4,6083e1cf-a636-4c64-9253-ac86cd08f5d7,3SC Production 2.3,NaN,3279,AF,terrafund,3SC Production 2.3,approved
6,617601e0-9839-49fd-b48e-6c07404e7140,Afram Headwaters Restoration Initiative (AHRI),NaN,1358,GH,terrafund-landscapes,1.\tSite Reconnaissance – which has already be...,approved
...,...,...,...,...,...,...,...,...
268,2d1cffcd-300f-4939-a136-310347cf6879,WITHDRAWN: Divine Bamboo Afri 100,NaN,953,UG,terrafund,NOTE: THIS PROJECT HAS WITHDRAWN FROM TERRAFUN...,approved
269,568dc331-b945-41fd-ab57-80a98be57941,Women and Youth-Led Climate Initiative for res...,NaN,1814,KE,terrafund-landscapes,2024\n1. Community sensitization and mobilizat...,approved
270,77fc2c03-5605-45b3-b417-f69d93157215,Women led Community forest Conservation Init...,NaN,2815,KE,terrafund-landscapes,Project Activity \tSummary\n1. Stakeholders in...,approved
271,96c86eae-d4f9-45d8-9780-69c55a9e36e9,World Vision Ethiopia One Tree Planted Project,NaN,1047,ET,terrafund,07/01/2022 to 07/31/2023,approved


In [7]:
# Create a list of project ids to query
ids = list(set(full.project_id))

# Short list for testing purposes
ids = ids[:11]

In [8]:
# Pull polygons from projects in list of ids from TerraMatch API
project_results = pull_tm_api_data(staging_url, headers, ids)

Processing Projects: 100%|██████████| 11/11 [00:11<00:00,  1.01s/project]


In [9]:
# Convert the polygon geometries into a dataframe
project_df = pd.DataFrame(project_results)
project_df.columns = project_df.columns.str.lower()

In [12]:
project_df

,name,status,siteid,geometry,plantstart,plantend,practice,targetsys,distr,numtrees,calcarea,indicators,establishmenttreespecies,reportingperiods,poly_id,project_id
0,Inziga02 (new),approved,2a2f156b-1954-43da-b3a4-165c8351e854,"{'type': 'Polygon', 'coordinates': [[[30.10687...",2022-06-20,None,tree-planting,agroforest,Null,NaN,14.511431,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2022-09-30T00:00:00.000Z', 'submit...",5d579f34-0993-4953-ace2-3c3ea4d9cb0e,d5e0a4ff-8601-45d0-9020-8c104e5ea508
1,Rwesororo02 (new),approved,f8abe452-1498-4d49-8c33-c11c1ade6b24,"{'type': 'Polygon', 'coordinates': [[[29.90255...",2022-05-04,None,tree-planting,agroforest,Null,NaN,38.940311,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2022-09-30T00:00:00.000Z', 'submit...",e28aa8ac-3da4-48aa-bb41-a43b0ed8319a,d5e0a4ff-8601-45d0-9020-8c104e5ea508
2,Kakindo04 (new),approved,961fbbb3-e3bd-48ed-a4e3-b6f02805c5e2,"{'type': 'Polygon', 'coordinates': [[[30.08562...",2022-05-04,None,tree-planting,agroforest,Null,NaN,2.189314,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2022-09-30T00:00:00.000Z', 'submit...",1c4f53d1-0a68-4846-a794-1ae87f280833,d5e0a4ff-8601-45d0-9020-8c104e5ea508
3,Bikone01 (new),approved,a4aff969-150f-444a-9397-7f3662dfc2d0,"{'type': 'Polygon', 'coordinates': [[[30.07313...",2022-05-04,None,tree-planting,agroforest,Null,NaN,19.800821,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2022-09-30T00:00:00.000Z', 'submit...",a0ae4e51-1b19-4ee9-a002-e6062774537a,d5e0a4ff-8601-45d0-9020-8c104e5ea508
4,Bikone02 (new),approved,a4aff969-150f-444a-9397-7f3662dfc2d0,"{'type': 'Polygon', 'coordinates': [[[30.07355...",2022-05-04,None,tree-planting,agroforest,Null,NaN,18.734166,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2022-09-30T00:00:00.000Z', 'submit...",2d5a13e0-8251-4d50-ae86-e1e3f7d3e7e9,d5e0a4ff-8601-45d0-9020-8c104e5ea508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,Mapiko,approved,6543989e-404e-4761-9ad3-c916e6bfc584,"{'type': 'Polygon', 'coordinates': [[[34.14233...",2022-11-01,None,assisted-natural-regeneration,natural-forest,None,NaN,1.006391,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2025-01-31T00:00:00.000Z', 'submit...",c7aa1996-6e65-4a0e-a406-4606f12aa278,16b297b3-30a3-4624-bcc9-4333919f66fc
118,Mtsinde_wa_dimba,approved,6543989e-404e-4761-9ad3-c916e6bfc584,"{'type': 'Polygon', 'coordinates': [[[34.14871...",2022-11-01,None,assisted-natural-regeneration,natural-forest,None,NaN,1.006389,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2025-01-31T00:00:00.000Z', 'submit...",8ff7f1d8-9913-44e9-9a0b-89195503a4af,16b297b3-30a3-4624-bcc9-4333919f66fc
119,Namilulu_source,approved,6543989e-404e-4761-9ad3-c916e6bfc584,"{'type': 'Polygon', 'coordinates': [[[34.16725...",2022-11-01,None,assisted-natural-regeneration,natural-forest,None,NaN,1.006371,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2025-01-31T00:00:00.000Z', 'submit...",71cfcb25-a7d7-4f59-bed0-0c5d3a0c6c46,16b297b3-30a3-4624-bcc9-4333919f66fc
120,Nanjewa,approved,6543989e-404e-4761-9ad3-c916e6bfc584,"{'type': 'Polygon', 'coordinates': [[[34.15959...",2022-11-01,None,assisted-natural-regeneration,natural-forest,None,NaN,1.006380,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2025-01-31T00:00:00.000Z', 'submit...",697955b2-c9e6-4a6a-97ca-b8339c463ccf,16b297b3-30a3-4624-bcc9-4333919f66fc


In [13]:
len(project_df.columns)

16

In [11]:
# Export the polygon geometries & metadata as a csv
project_df.to_csv('../data/tm_api_02122025.csv', index=False) # To the darby-tm-api-pull repo
project_df.to_csv('/home/darby/github_repos/darby-maxar-tools/data/tm_api_021225.csv', index=False) # To the darby-maxar-tools repo